In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df.index = pd.to_datetime(df.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]
display(df.head(1), df.tail(1))
df.shape


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2010-06-01 00:00:00+00:00,-0.195535,-0.431203,-0.701336,-0.140981,-0.470019,-0.252915,-0.504554,-0.271715,0.354467,0.168679,...,-0.372762,0.185407,-0.157289,-0.047979,-0.305857,-0.55999,-0.335956,0.293033,0.037194,-0.217444


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,-0.05858,-0.048583,0.019215,0.010852,-0.092832,0.009434,0.079595,0.068199,0.036857,-0.023727,...,0.065983,0.070777,-0.053466,-0.080965,-0.077221,-0.004721,-0.121889,0.041164,0.052337,0.119222


(74593, 28)

In [3]:
SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]



In [4]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[f'{sym}_tend'] = df[sym].rolling(rate).mean()
    data.index = df.index
    return data
        
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

In [5]:
rand = random_prediction(df)
mu = mean_prediction(df)

In [126]:
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

for sym in mu.columns:
    mu[sym] = mu[sym].map( lambda x: condition(x))
    
# mu.reset_index(inplace=True, drop=True)

display(rand.shape, mu.shape)


(74593, 28)

(74593, 28)

In [127]:
pred_last = rand.iloc[-120:]
pred = rand.iloc[-1:]
pred_last
pred = abs(pred)



In [128]:
def correlation_pairs(df, rate=120):
    
    data = pd.DataFrame([])
    for pair in df.columns:
        for second_pair in df.columns:
            data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(120).corr(df[second_pair])
    data.index = df.index
    data = data.iloc[-1:].T
    data.columns = ["correlation"]
    data = data[round(data["correlation"],6) != 1.000000]
    return data



def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    volatility = volatility.copy()
    volatility_T = volatility.T
    weight = volatility_T/ volatility_T.mean()
    return  weight

In [131]:
w = weighted_volatility(pred)
c = correlation_pairs(pred_last)

/tmp/ipykernel_1516/961547565.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(120).corr(df[second_pair])
/tmp/ipykernel_1516/961547565.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(120).corr(df[second_pair])
/tmp/ipykernel_1516/961547565.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

In [133]:
display(w.T,c.T)

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.979277,0.693818,0.866351,1.803933,1.192528,0.868928,1.089745,0.924965,0.845829,1.443402,...,1.438982,0.092826,1.554888,0.876313,0.199778,1.164805,1.71857,0.718804,0.408722,0.596769


,AUD_CAD_AUD_CHF_corr,AUD_CAD_AUD_JPY_corr,AUD_CAD_AUD_NZD_corr,AUD_CAD_AUD_USD_corr,AUD_CAD_CAD_CHF_corr,AUD_CAD_CAD_JPY_corr,AUD_CAD_CHF_JPY_corr,AUD_CAD_EUR_AUD_corr,AUD_CAD_EUR_CAD_corr,AUD_CAD_EUR_CHF_corr,...,USD_JPY_GBP_CHF_corr,USD_JPY_GBP_JPY_corr,USD_JPY_GBP_NZD_corr,USD_JPY_GBP_USD_corr,USD_JPY_NZD_CAD_corr,USD_JPY_NZD_CHF_corr,USD_JPY_NZD_JPY_corr,USD_JPY_NZD_USD_corr,USD_JPY_USD_CAD_corr,USD_JPY_USD_CHF_corr
correlation,0.031372,0.174553,-0.070898,0.03944,-0.021052,0.038249,-0.156459,0.000669,0.086874,0.080915,...,0.049999,0.042723,0.011753,0.093681,0.16271,-0.025366,0.044505,-0.111227,0.101073,0.041018


In [151]:
sym = 'AUD_JPY'
sym2 = 'AUD_CHF'

c.T.filter(regex=sym+'_'+sym2).values[0][0]

-0.09387557915357214

In [156]:
def expected_volatility(pred_vol, correlations):
    
    vols = pred_vol.copy()
    
    for sym in pred_vol.columns:
        
        real_vol = 0
        
        for sym2 in pred_vol.columns:
            
            if sym == sym2:
                real_vol += pred_vol[sym]
            else:
                real_vol += pred_vol[sym2] * correlations.filter(regex=sym+'_'+sym2).values[0][0]
                
        vols[sym] = round( real_vol.mean(), 4)
        
    total_vol = vols.sum(axis=1)[0]

    return total_vol, vols
    
        


In [159]:
t, v = expected_volatility(w.T,c.T)

print('Total volatilty:', t, '\nReal vols:\n', v)

Total volatilty: 30.2087 
Real vols:
                            AUD_CAD  AUD_CHF  AUD_JPY  AUD_NZD  AUD_USD  \
2022-07-01 00:00:00+00:00   1.0355   0.1573   0.1899    2.862   0.6754   

                           CAD_CHF  CAD_JPY  CHF_JPY  EUR_AUD  EUR_CAD  ...  \
2022-07-01 00:00:00+00:00   0.9197   0.3466   0.9814    0.635   2.4597  ...   

                           GBP_JPY  GBP_NZD  GBP_USD  NZD_CAD  NZD_CHF  \
2022-07-01 00:00:00+00:00    0.917  -1.0668   2.1183   1.9283   0.7511   

                           NZD_JPY  NZD_USD  USD_CAD  USD_CHF  USD_JPY  
2022-07-01 00:00:00+00:00   1.6756   1.6428    1.897   0.1661   1.3567  

[1 rows x 28 columns]
